In [ ]:
import numpy as np, keras, os, re, tensorflow as tf, scipy.misc
from PIL import Image
from keras import Model
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, BatchNormalization, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import TimeDistributed, Conv2D, MaxPooling2D, MaxPooling3D, Flatten, Activation, LeakyReLU, Input, Conv3D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# sort files as 1,2,3,4,... and not as 1,10,100,11,...
def sort_nicely(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort( key=alphanum_key )

In [ ]:
path = # path of saved .npy's

x_train, y_train = np.load(path+'data/x_train.npy',allow_pickle=True), np.load(path+'data/y_train.npy')
x_val,   y_val   = np.load(path+'data/x_val.npy',allow_pickle=True),   np.load(path+'data/y_val.npy')
x_test,  y_test  = np.load(path+'data/x_test.npy',allow_pickle=True),  np.load(path+'data/y_test.npy')

CLASSES = max(y_train)+1
SHAPE   = (175,175)
name    = path+'models/desired_name.hdf5'

In [ ]:
class DataGenerator(keras.utils.Sequence) :
  
    def __init__(self, images, labels, batch_size) :
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
    
    def __len__(self) :
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        
        batch_x = self.images[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        
        r = [batch_x[0]+'/'+f for f in os.listdir(batch_x[0])]; sort_nicely(r)
        ret = [np.array(Image.open(ri).resize(SHAPE))/255 for ri in r]
        
        return np.expand_dims(np.array(ret),0),  np.expand_dims(np.squeeze([to_categorical(batch_y,CLASSES)]*len(r)),0)

In [ ]:
def lstm_model(INPUT_SIZE,classes):
    video = Input(shape=(None,)+INPUT_SIZE)
    cnn_base = VGG16(input_shape=INPUT_SIZE,
                     weights="imagenet",
                     include_top=False)
    cnn_out = GlobalAveragePooling2D()(cnn_base.output)
    cnn = Model(input=cnn_base.input, output=cnn_out)
    cnn.trainable = False
    encoded_frames = TimeDistributed(cnn)(video)
    encoded_sequence = Bidirectional(LSTM(256,return_sequences=True))(encoded_frames)
    outputs = TimeDistributed(Dense(classes,activation="softmax"))(encoded_frames)
    model = Model([video], outputs)
    model.summary()
    return model

In [ ]:
def lstm_model(INPUT_SIZE,classes):

    shape = (None,) + INPUT_SIZE

    model = Sequential()
    model.add(Conv3D(32, kernel_size=(1,3,3),input_shape=shape))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.2))
    model.add(MaxPooling3D(pool_size=(1,2,2)))

    model.add(Conv3D(64, kernel_size=(1, 3, 3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.2))
    model.add(MaxPooling3D(pool_size=(1,2,2)))

    model.add(Conv3D(128, kernel_size=(1,3,3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.2))
    model.add(MaxPooling3D(pool_size=(1,2,2)))

    model.add(Conv3D(256, kernel_size=(1,3,3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.2))
    model.add(MaxPooling3D(pool_size=(1,2,2)))


    model.add(TimeDistributed(Flatten()))
    model.add(Bidirectional(LSTM(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(classes, activation='softmax')))

    model.summary()
    return model

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

config = tf.ConfigProto( device_count = {'GPU': 0})
config.gpu_options.allow_growth = True 
session = tf.Session(config=config)

In [ ]:
model = lstm_model(SHAPE+(3,),CLASSES)

In [ ]:
# SOS: Works only for batch_size = 1 !
train_generator    = DataGenerator(images=x_train, labels=y_train, batch_size=1)
val_generator      = DataGenerator(images=x_val,   labels=y_val,   batch_size=1)
test_generator     = DataGenerator(images=x_test,  labels=y_test,  batch_size=1)

In [ ]:
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        loss, acc = self.model.evaluate_generator(test_generator)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min',restore_best_weights=True)
mcp_save = ModelCheckpoint(name, save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
    
model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.SGD(),metrics=['accuracy'])
model.fit_generator(generator=train_generator,epochs=200,validation_data = val_generator,
                    verbose = 1,callbacks=[earlystop,mcp_save,reduce_lr_loss])

In [ ]:
model = keras.models.load_model(name)
model.evaluate_generator(test_generator)